In [28]:
import yfinance as yf
import pandas as pd
import math
import numpy as np

In [29]:
# If you don't have csv
# comment this and uncomment below cell
reliance = pd.read_csv("reliance.csv")
reliance.set_index('Date')

,Open,High,Low,Close,Volume
Date,,,,,
1996-01-01,10.471227,10.527414,10.402270,10.509537,48051995
1996-01-02,10.483998,10.535077,10.351192,10.427811,77875009
1996-01-03,10.598927,11.081625,10.483999,10.506985,96602936
1996-01-04,10.407379,10.440581,10.269465,10.409933,100099436
1996-01-05,10.369071,10.369071,10.249035,10.338424,76935930
...,...,...,...,...,...
2022-04-27,2755.850098,2828.000000,2755.050049,2778.350098,8924920
2022-04-28,2809.949951,2851.000000,2786.100098,2819.850098,11820892
2022-04-29,2856.149902,2856.149902,2777.300049,2790.250000,8406156


In [30]:
# reliance = yf.Ticker("RELIANCE.NS").history(period="max", actions=False)

In [31]:
reliance['boringCandle'] = (abs(reliance['Open']-reliance['Close']) / abs(reliance['High']-reliance['Low']) < 0.5)
reliance['greenExcitingCandle'] = (abs(reliance['Open']-reliance['Close'])/abs(reliance['High']-reliance['Low']) > 0.5) & (reliance['Open']<reliance['Close'])
reliance['redExcitingCandle'] = (abs(reliance['Open']-reliance['Close'])/abs(reliance['High']-reliance['Low']) > 0.5) & (reliance['Open']>reliance['Close'])